## This notebook generates LOS models for NH facilityfacts files ##

In [ ]:
import os
from collections import defaultdict
import pandas as pd
from IPython.display import display, HTML
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import expon, lognorm
import scipy.optimize as op
from stats import fullCRVFromPDFModel, CachedCDFGenerator
import tools_util as tu
import pyrheautils as pu
import phacsl.utils.formats.yaml_tools as yaml_tools
import json


## Load the model ##

In [ ]:
runDesc = '/home/welling/git/pyRHEA_github/src/sim/twoyear_allfac_OC.yaml'
inputDict = tu.readModelInputs(runDesc)
pu.prepPathTranslations(inputDict)
facDict = tu.getFacDict(inputDict)


## Load the line list data ##

In [ ]:
lineFName = 'OC_Nursing_Home_LOS_Line-Lists_for_RHEA_2.0_-_2011-2015_-_Adult_Only_-_06-10-2019_UPDATE_NH_LOS_Line_List.csv'
#                                      'OC_Nursing_Home_LOS_Line-Lists_for_RHEA_2.0_-_2011-2015_-_Adult_Only_-_05-21-2019_UPDATE_v2_NH_LOS_Line_List.csv'))
lineDF = pd.read_csv(pu.pathTranslate('$(MODELDIR)/%s' % lineFName))

print lineDF.columns
lineDF = lineDF[lineDF['NOT YET DISCHARGED?'].isnull()]
display(lineDF.head())

## Some Useful Functions ##

Also, make the plots big enough to see

In [ ]:
%config InlineBackend.print_figure_kwargs = {'bbox_inches':None}
#plt.rcParams["figure.figsize"] = [16, 10]
plt.rcParams["figure.figsize"] = [8, 5]

In [ ]:
def pltCRV(crv, scale=1.0, label=None, xlim=None):
    mv = crv.mean()
    if xlim is None:
        xV = np.linspace(0.0, round(3*mv), 101)
    else:
        xV = np.linspace(0.0, xlim, 101)
    yV = scale * crv.pdf(xV)
    if label is None:
        plt.plot(xV, yV, '-')
    else:
        plt.plot(xV, yV, '-', label=label)
    if xlim is None or mv <= xlim:
        plt.plot([mv], [0.0], '*')
    print mv


In [ ]:
%config InlineBackend.print_figure_kwargs = {'bbox_inches':None}
%matplotlib inline
plt.rcParams["figure.figsize"] = [16, 10]

showL = np.random.choice([abbrev for abbrev, rec in facDict.items() if rec['category'] == 'NURSINGHOME'], size=5)
print showL
for abbrev in showL:
    facLOSDF = lineDF[lineDF['NH CODE'] == abbrev]
    histoD = defaultdict(int)
    for idx, row in facLOSDF.iterrows():
        los = int(row['ADJUSTED LOS (days)'])
        histoD[los] += 1
    losModel = {'pdf': 'empirical(loc=$0,scale=$1)',
               'prov': 'experiment',
               'parms': [0.0, 1.0],
               'sampleHistogram': histoD}
    crv = fullCRVFromPDFModel(losModel)
    plt.hist(facLOSDF['RAW LOS (days)'], range=(0,300), bins=300, label='adjusted line-list')
    nLines = facLOSDF.count()['NH CODE']
    pltCRV(crv, scale=nLines/crv.cdf(365.), label='empirical pdf', xlim=300)
    plt.legend()
    #plt.title('%s %s' % (abbrev, fitLOSModel['parms']))
    plt.title('%s samples and empirical pdf' % abbrev)
    plt.show()


In [ ]:
plt.rcParams["figure.figsize"] = [16, 10]

for abbrev in showL:
    facLOSDF = lineDF[lineDF['NH CODE'] == abbrev]
    histoD = defaultdict(int)
    for idx, row in facLOSDF.iterrows():
        los = int(row['ADJUSTED LOS (days)'])
        histoD[los] += 1
    losModel = {'pdf': 'empirical(loc=$0,scale=$1)',
               'prov': 'experiment',
               'parms': [0.0, 1.0],
               'sampleHistogram': histoD}
    crv = fullCRVFromPDFModel(losModel)
    nsamps = 1000
    plt.hist(crv.rvs(size=nsamps), range=(0,300), bins=300, label='drawn samples')
    pltCRV(crv, scale=nsamps/crv.cdf(365.), label='empirical pdf', xlim=300)
    plt.legend()
    #plt.title('%s %s' % (abbrev, fitLOSModel['parms']))
    plt.title('%s empirical pdf and samples drawn from empirical distribution' % abbrev)
    plt.show()


In [ ]:
plt.rcParams["figure.figsize"] = [16, 2]
showL = ['STAN']

for abbrev in showL:
    facLOSDF = lineDF[lineDF['NH CODE'] == abbrev]
    histoD = defaultdict(int)
    for idx, row in facLOSDF.iterrows():
        los = int(row['ADJUSTED LOS (days)'])
        histoD[los] += 1
    losModel = {'pdf': 'empirical(loc=$0,scale=$1)',
               'prov': 'experiment',
               'parms': [0.0, 1.0],
               'sampleHistogram': histoD}
    crv = fullCRVFromPDFModel(losModel)
    nsamps = 1000
    plt.hist(crv.rvs(size=nsamps), range=(0,200), bins=300, label='drawn samples')
    pltCRV(crv, scale=nsamps/crv.cdf(365.), label='empirical pdf', xlim=200)
    plt.legend()
    #plt.title('%s %s' % (abbrev, fitLOSModel['parms']))
    plt.title('%s empirical pdf and samples drawn from empirical distribution' % abbrev)
    plt.show()


## Save the empirical models ##

In [ ]:
newFacDict = {}
for fac, rec in facDict.items():
    if rec['category'] == 'NURSINGHOME':
        print 'processing %s' % fac
        facLOSDF = lineDF[lineDF['NH CODE'] == abbrev]
        histoD = defaultdict(int)
        for idx, row in facLOSDF.iterrows():
            los = int(row['ADJUSTED LOS (days)'])
            histoD[los] += 1
        histoD = {str(k) : v for k, v in histoD.items()} # get rid of defaultdict, use strings as keys
        losModel = {'pdf': 'empirical(loc=$0,scale=$1)',
                   'prov': 'empirical based on %s' % lineFName,
                   'parms': [0.0, 1.0],
                   'sampleHistogram': histoD}
        newRec = rec.copy()
        newRec['losModel'] = losModel
        newFacDict[fac] = newRec

outDir = pu.pathTranslate('$(MODELDIR)/facilityfactsUpdated')
print 'writing yaml files to %s' % outDir
yaml_tools.save_all(outDir, newFacDict.values())

## Read and plot intervalprob values captured by JournalingCachedCDFGenerator ##

In [ ]:
simDF = pd.read_msgpack(pu.pathTranslate('$(SIMDIR)/cdf_intervalprob_results.mpz'))
#display(simDF.head())
for abbrev in simDF['abbrev'].unique():
    print abbrev
    df = simDF[simDF.abbrev == abbrev]
    #display(df.head())
    for pdf in df['pdf'].unique():
        print pdf
        subDF = df[df.pdf == pdf]
        display(subDF.tail())
        plt.plot(0.5*(subDF['start'] + subDF['end']), subDF['rslt'], '*', alpha=0.01, label=pdf)
    plt.xlim(0.0, 300.0)
    plt.title(abbrev)
    plt.legend()
    plt.show()
    

## Some experiments in storing samples as yaml ##

In [ ]:
from collections import defaultdict
facLOSDF = lineDF[lineDF['NH CODE'] == 'NEWO']
sampDF = facLOSDF['RAW LOS (days)']
dct = defaultdict(int)
for elt in sampDF:
    dct[elt] += 1
dct = {k: v for k, v in dct.items()}
print dct

   

In [ ]:
#shortHS = pd.Series(losHV)
mystr = json.dumps(dct)
print 'length: ', len(mystr)
print mystr


In [ ]:
import json
dct = json.loads(mystr)
idct = {}
maxK = 0
for k, v in dct.items():
    ik = int(k)
    idct[ik] = v
    maxK = max(maxK, ik)
print maxK
print idct


In [ ]:
arr = np.zeros(maxK+1, dtype=np.int)
for k, v in idct.items():
    arr[k] = v
print len(arr)
print arr

In [ ]:
sumArr = np.cumsum(arr)
print len(sumArr)
print sumArr

In [ ]:
def intervalprob(day, sumArr):
    if day < len(sumArr):
        prevCt = 0 if day == 0 else sumArr[day-1]
        delta = sumArr[day] - prevCt
    else:
        prevCt = 0
        delta = 0
    return float(delta) / float(sumArr[-1] - prevCt)

In [ ]:
print intervalprob(37, sumArr)

In [ ]:
xV = np.arange(0.0, 300.0, 1.0)
yV = [intervalprob(int(x), sumArr) for x in xV]
plt.plot(xV, yV, '-')
plt.show()

In [ ]:
import pickle
dct = {(abbrev, 'NURSING'): rec['meanPop']['value'] for abbrev, rec in facDict.items() if rec['category']=='NURSINGHOME'}
with open('/home/welling/git/pyRHEA_github/src/sim/oc_expected_pop.pkl', 'w') as f:
    pickle.dump(dct,f)

In [ ]:
lst = [(rec['meanLOS']['value'], rec['abbrev']) for rec in facDict.values() if rec['category'] == 'NURSINGHOME']
lst = lst[:]
lst.sort()
print lst